<a href="https://colab.research.google.com/github/Rabita2005/weatherwise-rabita-sami/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🧰 Setup and Imports



In [2]:
!pip install fetch-my-weather
!pip install hands-on-ai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 68.9 MB/s eta 0:00:00
  Attempting uninstall: fastcore
    Found existing in

In [4]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

Enter your API key: 28c24ec30e34fdfd8879b0b363bdc832


## 📦 Setup and Configuration
Import required packages and setup environment.

In [7]:
# Install required packages if running in Colab
# !pip install fetch-my-weather ipywidgets matplotlib

import os
import json
import ipywidgets as widgets
from IPython.display import display, clear_output
import fetch_my_weather
import matplotlib.pyplot as plt

## 🌤️ Weather Data Functions

In [8]:
def get_weather_data(location):
    try:
        return fetch_my_weather.get_weather(location=location, units="m")
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return None

## 📊 Visualisation Functions

In [9]:
def create_temperature_visualisation(weather_data):
    if not weather_data or not hasattr(weather_data, "weather"):
        print("No forecast data available.")
        return

    days = [day.date for day in weather_data.weather]
    temps = [day.maxtempC for day in weather_data.weather]

    plt.figure(figsize=(8,4))
    plt.plot(days, temps, marker='o', color='orange')
    plt.title("Temperature Forecast")
    plt.xlabel("Date")
    plt.ylabel("Max Temperature (°C)")
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [10]:
def create_precipitation_visualisation(weather_data):
    if not weather_data or not hasattr(weather_data, "weather"):
        print("No forecast data available.")
        return

    days = [day.date for day in weather_data.weather]
    rain_chance = [int(day.hourly[0].chanceofrain or 0) for day in weather_data.weather]

    plt.figure(figsize=(8,4))
    plt.bar(days, rain_chance, color='skyblue')
    plt.title("Chance of Rain (%)")
    plt.xlabel("Date")
    plt.ylabel("Rain Chance (%)")
    plt.ylim(0, 100)
    plt.grid(axis='y')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 🤖 Natural Language Processing

In [11]:
def parse_weather_question(question):
    question = question.lower()
    parsed = {"location": None, "attribute": None}

    if "temperature" in question or "hot" in question:
        parsed["attribute"] = "temperature"
    elif "rain" in question or "precipitation" in question:
        parsed["attribute"] = "precipitation"

    if "in " in question:
        parsed["location"] = question.split("in ")[-1].split()[0]

    return parsed

## 🧭 User Interface

In [12]:
location_input = widgets.Text(
    value='Perth',
    placeholder='Enter a city',
    description='Location:',
    disabled=False
)

ask_button = widgets.Button(description="Check Weather")
output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output()
        location = location_input.value
        data = get_weather_data(location)
        if data:
            print(f"Weather forecast for {location.title()}:")
            create_temperature_visualisation(data)
            create_precipitation_visualisation(data)
        else:
            print("Could not retrieve weather data. Please try a different location.")

ask_button.on_click(on_button_click)

display(location_input, ask_button, output)

Text(value='Perth', description='Location:', placeholder='Enter a city')

Button(description='Check Weather', style=ButtonStyle())

Output()

## 🧩 Main Application Logic

In [13]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [15]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.